# Circos plot of the Leiden clusters and mechanism of action distribution

Loading the R packages.

In [1]:
suppressWarnings({suppressPackageStartupMessages({
    library(Seurat)
    library(ggsci)
    library(reshape2)
    library(readxl)
    library(viridis)
    library(circlize)
})})

Loading the data.

In [2]:
hgsoc <- readRDS("HGSOC_CellHashing_CLUSTERED.RDS")

Extracting the data needed for the circos plot.

In [3]:
clusters_drugs_batch <- hgsoc@meta.data[, c(6, 26)]
head(clusters_drugs_batch)

,Treatment_group,RNA_clusters
,<chr>,<fct>
JHOS2_AAACCCAAGCAAATCA-1,Fedratinib,5
JHOS2_AAACCCAAGGTCGCCT-1,BMS-754807,8
JHOS2_AAACCCACAGAACATA-1,SCH772984,2
JHOS2_AAACCCAGTCTGTCAA-1,TGX-221,2
JHOS2_AAACCCAGTCTGTGGC-1,Ipatasertib,2
JHOS2_AAACCCAGTTGGACCC-1,Ralimetinib,7


Adding “RNA cluster” in front of the number of the cluster.

In [4]:
clusters_drugs_batch$RNA_clusters <- paste0("RNA cluster ", clusters_drugs_batch$RNA_clusters)

Loading the mechanisms of actions/higher annotations and adding their info.

In [5]:
high_anno <- as.data.frame(read_xlsx(path = "mechanisms_of_actions.xlsx", sheet = 1))
high_anno <- high_anno[, c(1:3)]
rownames(high_anno) <- high_anno$`Preferred name`

clusters_drugs_batch$higher_level_mechanism <- sapply(clusters_drugs_batch$Treatment_group, 
                                                    function(x) high_anno[x, "Higher level classification"])

New names:
• `` -> `...4`
• `` -> `...5`


Fixing RNA clusters' and mechanisms' levels.

In [6]:
clusters_drugs_batch$RNA_clusters <- factor(clusters_drugs_batch$RNA_clusters,
                                           levels = paste0("RNA cluster ", rev(c(1:13))))

clusters_drugs_batch$higher_level_mechanism <- factor(clusters_drugs_batch$higher_level_mechanism,
                                                     levels = c("Control", "Bcl-2 inhibitor", "BET inhibitor", "CDK inhibitor",
                                                               "CHK1 inhibitor", "HDAC inhibitor", "IAPs/SMAC mimetic",
                                                               "Multi-kinase inhibitor", "PAK inhibitor", "PARP inhibitor",
                                                               "PI3K/mTOR/AKT inhibitor", "PLK1 inhibitor", "Ras/Raf/MEK/ERK inhibitor",
                                                               "XPO1/CRM1 inhibitor"))   

In [7]:
clusters_drugs_batch <- clusters_drugs_batch[order(clusters_drugs_batch$higher_level_mechanism, decreasing = T), ]
clusters_drugs_batch <- clusters_drugs_batch[order(clusters_drugs_batch$RNA_clusters, decreasing = T),]

Colors will be set with Adobe Illustrator.

In [8]:
pdf("Circos_RNAclusters_model_newIntegration.pdf", width = 8, height = 8)
chordDiagram(x = clusters_drugs_batch[, c("RNA_clusters", "higher_level_mechanism")], 
             annotationTrack =  "grid", 
             column.col = custom_palette, 
             annotationTrackHeight = c(0.05, 0.01), 
             transparency = 0.1, scale = FALSE,
             preAllocateTracks = list(track.height = max(strwidth(c(clusters_drugs_batch$RNA_clusters, 
                                                                    clusters_drugs_batch$higher_level_mechanism)))))
# we go back to the first track and customize sector labels
circos.track(track.index = 1, panel.fun = function(x, y) {
    circos.text(CELL_META$xcenter, CELL_META$ylim[1], CELL_META$sector.index, 
        facing = "clockwise", niceFacing = TRUE, adj = c(0, 0.5))
}, bg.border = NA) # here set bg.border to NA is important) # Border color)
circos.clear()
dev.off()

png 
  2

In [9]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Rocky Linux 8.8 (Green Obsidian)

Matrix products: default
BLAS/LAPACK: /homedir01/adini22/.conda/envs/cellhashing_analyses/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] circlize_0.4.15    viridis_0.6.2      viridisLite_0.4.1  readxl_1.4.1      
[5] reshape2_1.4.4     ggsci_3.0.0        SeuratObject_4.1.3 Seurat_4.3.0.9002 

loaded via a namespace (and not attached):
  [1] Rtsne_0.16             colorspace_2.1-0       deldir_1.0-6      